<a href="https://colab.research.google.com/github/Hoda1394/brain_hack/blob/master/Predict_on_blocks_by_kwyk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O- http://neuro.debian.net/lists/bionic.us-nh.full | tee /etc/apt/sources.list.d/neurodebian.sources.list \
 && apt-key adv --recv-keys --keyserver hkp://pool.sks-keyservers.net:80 0xA5D32F012649A5A9 \
 && apt-get update \
 && apt-get install git-annex-standalone git

In [ ]:
!git-annex version

In [ ]:
!pip install --no-cache-dir nilearn datalad datalad-osf
!pip install --no-cache-dir https://github.com/neuronets/nobrainer/tarball/master

In [ ]:
import nobrainer
!datalad clone https://github.com/neuronets/trained-models && \
  cd trained-models && git-annex enableremote osf-storage && \
  datalad get -s osf-storage neuronets/kwyk/0.4.1/all_50_bvwn_multi_prior

In [ ]:
import tensorflow as tf
import nibabel as nib

In [ ]:
# prepare your data 
# upload your data
# use functions nib.processing.conform to conform the size to 32 block




image_path = ""

In [ ]:
# load the model
model_path = "trained-models/neuronets/kwyk/0.4.1/all_50_bvwn_multi_prior/"
model = tf.saved_model.load(model_path)
model = model.signatures['serving_default']

In [ ]:
# predict from an input block

from nobrainer.utils import StreamingStats
import numpy as np

img = nib.load(filepath)
inputs = np.asarray(img.dataobj)
img.uncache()
inputs = inputs.astype(np.float32)

print("Normalizer being used {n}".format(n = normalizer))
    if normalizer:
        features = normalizer(inputs)
        print(features.mean())
        print(features.std())
    else:
        features = inputs

# Add a dimension for single channel.
features = features[..., None]
data = tf.convert_to_tensor(features)
prediction = model(data)

out = prediction['probabilities']
print("output size",out.shape)

# return in the form of a nifti image
output = nib.spatialimages.SpatialImage(
          dataobj=out, affine=img.affine, header=img.header, extra=img.extra)

In [ ]:
# resize back the output image by nib.processing.resample_from_to or any other tool





In [ ]:
# plot output with nilearn or just numpy
from nilearn import plotting

plotting.plot_roi(out, bg_img=image_path, alpha=0.4)